In [1]:
import pandas as pd
df = pd.read_csv("regression_exercise.csv")
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [2]:
# creating target variable
y = df["Item_Outlet_Sales"]
X = df.drop(["Item_Outlet_Sales","Item_Identifier"],axis = 1)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [7]:
categorical_cols = [cname for cname in X_train_full.columns if
                        X_train[cname].dtype == "object"]

In [9]:
numerical_cols = [cname for cname in X_train_full.columns if X_train[cname].dtype in ['int64', 'float64']]

In [11]:
X_train_cat = X_train[categorical_cols]

In [13]:
X_train_num = X_train[numerical_cols]

In [22]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(strategy='mean')
imp_most_frequent = SimpleImputer(strategy='most_frequent')

In [16]:
X_train_num = imp_mean.fit_transform(X_train_num)

In [19]:
X_train_num.shape

(6818, 4)

In [23]:
X_train_cat = imp_most_frequent.fit_transform(X_train_cat)

In [26]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X_train_cat_dummied = enc.fit_transform(X_train_cat)

In [27]:
X_train_cat_dummied.shape

(6818, 41)

In [30]:
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

class ToDenseTransformer():

    # here you define the operation it should perform
    def transform(self, X, y=None, **fit_params):
        return X.todense()

    # just return self
    def fit(self, X, y=None, **fit_params):
        return self

# need to make matrices dense because PCA does not work with sparse vectors.
pipeline = Pipeline([
    ('to_dense',ToDenseTransformer()),
    ('pca',PCA(n_components=3))])

In [32]:
X_train_cat_pca = pipeline.fit_transform(X_train_cat_dummied)

In [36]:
type(X_train_cat_pca)

numpy.ndarray

In [38]:
from sklearn.feature_selection import SelectKBest
X_train_num_kbest = SelectKBest(k=3).fit_transform(X_train_num, y_train)

In [39]:
X_train_num_kbest.shape

(6818, 3)

In [41]:
import numpy as np
X_train_preprocessed = np.column_stack((X_train_cat_pca, X_train_num_kbest))

In [42]:
X_train_preprocessed.shape

(6818, 6)

In [51]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Use base_model in Task I
base_model = RandomForestRegressor()

In [52]:
base_model.fit(X_train_preprocessed, y_train)

RandomForestRegressor()

In [53]:
base_model.score(X_train_preprocessed, y_train)

0.937327345760116